In [ ]:
train_tfms = T.Compose([  # train transform(augmentation 포함)
    T.RandomCrop(32, padding=4),  # 32x32를 padding 후 random crop
    T.RandomHorizontalFlip(),  # 좌우 반전
    T.ToTensor(),  # PIL -> torch tensor (C,H,W), 0~1
    T.Normalize(CIFAR10_MEAN, CIFAR10_STD),  # 정규화, -2.5~2.5
])

test_tfms = T.Compose([  # test transform(augmentation 없음)
    T.ToTensor(),  # 텐서 변환
    T.Normalize(CIFAR10_MEAN, CIFAR10_STD),  # 정규화
])

data_root = './data'  # 데이터 저장 경로
train_set = torchvision.datasets.CIFAR10(root=data_root, train=True, download=True, transform=train_tfms)  # train set
test_set  = torchvision.datasets.CIFAR10(root=data_root, train=False, download=True, transform=test_tfms)  # test set

class_names = train_set.classes  # 클래스 이름 목록, 이런식으로 바로 접근 가능한 경우도 있으니 체크는 해둘것.

In [ ]:
# CIFAR-10에서 널리 쓰는 normalize 값
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)  # 채널별 평균(R,G,B)
CIFAR10_STD  = (0.2023, 0.1994, 0.2010)  # 채널별 표준편차(R,G,B)

def denorm(x): # -2.5~2.5
    mean = torch.tensor(CIFAR10_MEAN).view(1,3,1,1)  # (1,3,1,1) 형태로 broadcast
    std  = torch.tensor(CIFAR10_STD).view(1,3,1,1)  # (1,3,1,1) 형태로 broadcast
    return x * std + mean  # 정규화 역변환, 0~1
    # 이렇게 하면 shape이 (batch_size, 3, height, width)와 브로드캐스팅이 맞는단다.
    # 즉, 각 채널별 평균값이 배치 전체와 모든 픽셀 위치에 자동으로 확장된다고

In [ ]:
 # 파라미터 개수 체크
num_params = sum(p.numel() for p in model.parameters()) 

In [ ]:
# 사전학습 모델 로드
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)  # 가중치 포함 pretrained DETR-ResNet50 로드

In [ ]:
# 가장 큰 logit의 클래스 선택, 
def accuracy_top1(logits, targets):
    preds = logits.argmax(dim=1)  # logits = [B, C], preds = [B], dim=1이유는 dim=0은 batch 차원이라서
    return (preds == targets).float().mean().item()  # top-1 accuracy, scalar

In [ ]:
optimizer.zero_grad(set_to_none=True)  # gradient 초기화
# zero_grade는 모든 파라미터의 gradient를 초기화 해주는 역할
# 다음 backward 시점에 새 텐서를 할당하게 됨. 불필요한 0 초기화 연산이 사라짐.
# grad=0일 때는 gradient가 항상 존재하지만 값이 0.
# grad=None일 때는 gradient가 아직 계산되지 않은 상태로 취급됩니다.
# backward가 호출되면 새 gradient 텐서가 생성됩니다.

In [ ]:
""" 모델 안에서 특정 이름으로 지정된 모듈 찾아 반환 """
def get_module_by_name(model: nn.Module, name: str) -> nn.Module:
    cur = model  # 시작 모듈
    for part in name.split('.'):  # 점(.) 기준으로 순회
        if part.isdigit():
            cur = cur[int(part)]  # Sequential 인덱스 접근
        else:
            cur = getattr(cur, part)  # attribute 접근
    return cur

In [ ]:
def capture_activation(model, layer_name, x_single):
    layer = get_module_by_name(model, layer_name)  # target layer
    activ = {}  # 저장용 dict
    def hook(m, i, o):  # callback function
        activ['feat'] = o.detach().cpu()  # (B,C,H,W) 저장

    h = layer.register_forward_hook(hook)  # target layer처리시 forward hook 등록 (hook 함수 실행)
    model.eval()  # eval 모드
    with torch.no_grad():
        _ = model(x_single)  # forward 수행
    h.remove()  # hook 제거
    assert 'feat' in activ, f"Hook failed for layer {layer_name}"  # 방어 코드
    return activ['feat'][0]  # (C,H,W) 반환

In [ ]:
def show_topk_activation_grid(feat_chw, title, topk=16):
    ch_scores = feat_chw.abs().mean(dim=(1,2))  # 채널별 평균 크기 점수
    idx = torch.topk(ch_scores, k=min(topk, feat_chw.shape[0])).indices  # Top-K 채널 인덱스
    maps = []  # (K,1,H,W)로 만들 리스트
    for ci in idx:
        m = feat_chw[int(ci)]  # (H,W)
        m = (m - m.min()) / (m.max() - m.min() + 1e-6)  # 0~1 정규화
        maps.append(m[None, None, ...])  # (1,1,H,W)
    maps = torch.cat(maps, dim=0)  # (K,1,H,W)
